# Shared File System

- **Estimated time**: 30 minutes
- **Requirements**:

In this tutorial we'll learn how to use the shared file system. For other data management strategies, see [Data management in the Chameleon Cloud](https://github.com/ChameleonCloud/notebooks/blob/master/tutorials/getting-started/DataManagement.ipynb).

## Tutorial

1. [Create a share](#Step-1:-Create-a-share)
1. [Reserve resources](#Step-2:-Reserve-resources)
1. [Start a server](#Step-3:-Start-a-server)
1. [Talk to the server](#Step-4:-Talk-to-the-server)
1. [View share and check access rules](#Step-5:-View-share-and-check-access-rules)
1. [Mount the share](#Step-6:-Mount-the-share)
1. [Clean up](#Step-7:-Clean-up)

### Getting started

Before you do anything, you should first set up a few variables.

In [ ]:
import chi

project_name = "chameleon" # Replace with your project name
region_name = "CHI@UC"     # Replace site if not use CHI@UC

chi.set('project_name', project_name)
chi.set('region_name', region_name)
chi.use_site(region_name)

NAME = "chi-share-tutorial" # a name for all openstack components

## Step 1: Create a share

In this step, you will create a 1 GiB share. A share is a pre-allocated storage space at a CephFS.

**Note**: We do not charge SUs for the storage spaces of your shares. However, we do limit the total size and the number of shares you can create within your project. The maximum number of shares is 10 and the maximum size allowed for all shares in a project is 2000 GiB.

In [ ]:
import chi.share

my_share = chi.share.create_share(size=1, name=NAME)
my_share

## Step 2: Reserve resources

To provide isolation among shares created by different projects, accessing a share requires a storage network, which are special networks you can reserve to use.

In this tutorial, you will need 1 bare metal node, 1 storage network, and 1 floating IP.

In [ ]:
import chi.lease

# Create reservations for 1 node, 1 storage network, and 1 floating IP
reservations = []
chi.lease.add_node_reservation(reservations, node_type='compute_cascadelake_r')
chi.lease.add_network_reservation(reservations, network_name=NAME, resource_properties=["==", "$usage_type", "storage"])
chi.lease.add_fip_reservation(reservations, count=1)

# Create a lease
lease = chi.lease.create_lease(NAME, reservations=reservations)
lease
chi.lease.wait_for_active(lease['id'])  # Ensure lease has started

## Step 3: Start a server

In this step, you will launch a server on the reserved storage network.

In [ ]:
import chi.server

# Create a server
compute_reservation_id = [reservation for reservation in lease['reservations'] if reservation['resource_type'] == 'physical:host'][0]['id']
server = chi.server.create_server(NAME, 
                                  reservation_id=compute_reservation_id, 
                                  network_name=NAME, 
                                  image_name='CC-Ubuntu20.04')
# Wait until the server is active
chi.server.wait_for_active(server.id)

## Step 4: Talk to the server

To attach floating IP to your instance created on a storage network, you need to create a router with `public` external network. Then connect the storage subnet to the router. You must specify an unused IP address which belongs to the selected subnet.

In [ ]:
import chi.lease
import chi.network
import chi.server
from chi.ssh import Remote

# Get storage network id
network = chi.network.get_network(NAME)
network_id = network['id']

# Get an unused IP address on the storage subnet and create a port
subnet_id = chi.network.get_subnet_id(NAME + '-subnet')
port = chi.network.create_port(NAME, network_id, subnet_id=subnet_id)

# Create a router with public external network
router = chi.network.create_router(NAME, gw_network_name='public')

# Added port to router
chi.network.add_port_to_router_by_name(NAME, NAME)

# Attach reserved floating IP
floating_ip_address = chi.lease.get_reserved_floating_ips(lease['id'])
floating_ip = chi.server.associate_floating_ip(server.id, floating_ip_address=floating_ip_address[0])
chi.server.wait_for_tcp(floating_ip, 22)

# Create a remote connection
remote = Remote(floating_ip)

## Step 5: View share and check access rules

The paths of the export locations are important as you will use this path to mount your share to your bare metal instance. Also, the accessibility of the shares are controlled internally by the reservation service. You need to check if the access rules are granted to the share.

In [ ]:
import chi.network
import chi.share

share = chi.share.get_share(my_share.id)

# Get export path
export_path = share.export_locations[0]

# Get and check access rules
subnet = chi.network.get_subnet(NAME + '-subnet')
access_rules = chi.share.get_access_rules(share.id)
access_rule_found = False
for rule in access_rules:
    if rule.state == "active" and rule.access_to == subnet['cidr'] and rule.access_level == "rw":
        access_rule_found = True
        print("Access rule has been added successfully!")
        break
if not access_rule_found:
    print("Failed to find the access rule!")

## Step 6: Mount the share

Mounting your share to your instance is simple with the `mount` command.

In [ ]:
from chi.ssh import Remote

mnt_dir = "/mnt"

# Mount to mnt_dir 
remote.sudo(f"mount -t nfs -o nfsvers=4.2,proto=tcp {export_path} {mnt_dir}", hide=True)

# Add a file to share
remote.sudo(f"bash -c 'echo \"this is my test file\" > {mnt_dir}/mytext.txt'", hide=True)

# List mnt_dir
list_files = remote.sudo(f"ls -la {mnt_dir}", hide=True)
print(list_files.stdout)

# Unmount
remote.sudo(f"umount {mnt_dir}", hide=True)

## Step 7: Clean up

Clean up everything created in this tutorial.

In [ ]:
import chi.lease
import chi.network
import chi.share

# Delete lease
chi.lease.delete_lease(lease['id'])

# Delete router to public
chi.network.delete_router(router['id'])

# Delete share
chi.share.delete_share(share.id)

## Recap

You should know how to create a share and access your share using storage network. You should also learn how to mount a share to your instance. Finally, you should have gained some familiarity with the `python-chi` library.

### Further reading

- [Chameleon user documentation](https://chameleoncloud.readthedocs.io/en/latest/technical/shares.html)
- [python-chi](https://python-chi.readthedocs.io/)
